# Project: ICD-AIS conversion using Deep Learning utilizing ICD10

This script translates the ICD codes into AIS codes using AAAM's official map.  The map does not get exact codes, but rather the chapter, body region, and severity.  A dummy predot code is formed from this data.

## Setup

In [1]:
import numpy as np
import pandas as pd

#### Files

In [2]:
# test ICD codes and demographics
test_icd_file = "../Data/NTDB_combine/ntdb_test_icd.csv"
test_demo_file = "../Data/NTDB_combine/ntdb_test_demo.csv"

# aaam map
aaam_map_file = "../Tools/AAAM ICD-ISS Map v2.0 (00082).xlsx"

# output of translation
results_ais_file = "../Results/Translations/ais_aaam_map.csv"

## Load data

In [3]:
# load demographics
test_demo = pd.read_csv(test_demo_file)

In [4]:
# load icd data
icd = pd.read_csv(test_icd_file)

In [5]:
icd.head()

,inc_key,ICDDIAGNOSISCODE
0,190026915434,S00.83XA
1,190026915434,S42.413A
2,190026915434,S71.039A
3,190026915434,S71.039A
4,190026915434,S71.139A


In [6]:
icd.inc_key

0        190026915434
1        190026915434
2        190026915434
3        190026915434
4        190026915434
             ...     
39450    190046127989
39451    190046127989
39452    190046128077
39453    190046128321
39454    190046128321
Name: inc_key, Length: 39455, dtype: int64

In [7]:
# load AAAM map
aaam_map = pd.read_excel(aaam_map_file, sheet_name="icd10Map", header=0, converters={'CODE':str})

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
aaam_map.head()

,CODE,DESCRIPTION,HIGHEST AIS SEVERITY,ISS BODY REGION,AIS CHAPTER
0,S00,Superficial injury of head,1,6,1
1,S00.0,Superficial injury of scalp,1,6,1
2,S00.00,Unspecified superficial injury of scalp,1,6,1
3,S00.00XA,"Unspecified superficial injury of scalp, initi...",1,6,1
4,S00.01,Abrasion of scalp,1,6,1


## Translate codes

In [9]:
# left join map to icd codes
mapped = icd.merge(aaam_map, left_on='ICDDIAGNOSISCODE', right_on='CODE', how='left')

In [10]:
mapped.head()

,inc_key,ICDDIAGNOSISCODE,CODE,DESCRIPTION,HIGHEST AIS SEVERITY,ISS BODY REGION,AIS CHAPTER
0,190026915434,S00.83XA,S00.83XA,"Contusion of other part of head, initial encou...",1.0,6.0,1.0
1,190026915434,S42.413A,S42.413A,Displaced simple supracondylar fracture withou...,2.0,4.0,7.0
2,190026915434,S71.039A,S71.039A,"Puncture wound without foreign body, unspecifi...",1.0,6.0,8.0
3,190026915434,S71.039A,S71.039A,"Puncture wound without foreign body, unspecifi...",1.0,6.0,8.0
4,190026915434,S71.139A,S71.139A,"Puncture wound without foreign body, unspecifi...",1.0,6.0,8.0


## Examine unmapped codes

In [11]:
# count unmapped icd codes
len(mapped[mapped.CODE.isna()])

1163

In [12]:
# percent unmapped icd codes
len(mapped[mapped.CODE.isna()]) / len(mapped)

0.029476618932961603

In [13]:
# most common ummapped codes
mapped[mapped.CODE.isna()].ICDDIAGNOSISCODE.value_counts()[:10].index.tolist()

['T14.8XXA',
 'T07.XXXA',
 'None',
 'F17.210',
 'I10',
 'N17.9',
 'E78.5',
 'D62',
 'E03.9',
 'R04.0']

## Clean up dataframe

In [14]:
# drop unmatched codes
mapped = mapped[~mapped.CODE.isna()]

In [15]:
# rename columns
mapped = mapped.rename(columns={'CODE':'icd','HIGHEST AIS SEVERITY':'severity','ISS BODY REGION':'region','AIS CHAPTER':'chapter'})

In [16]:
# make dummy predot code
mapped['ais_predot'] = (mapped.chapter * 100_000 + mapped.region * 10_000).astype(int)

In [17]:
# make full code
mapped['aiscode'] = mapped.ais_predot + (mapped.severity/10)

In [18]:
# select necessary columns
mapped = mapped[['inc_key','aiscode','ais_predot','severity','chapter','region']]

In [19]:
mapped.head()

,inc_key,aiscode,ais_predot,severity,chapter,region
0,190026915434,160000.1,160000,1.0,1.0,6.0
1,190026915434,740000.2,740000,2.0,7.0,4.0
2,190026915434,860000.1,860000,1.0,8.0,6.0
3,190026915434,860000.1,860000,1.0,8.0,6.0
4,190026915434,860000.1,860000,1.0,8.0,6.0


## Add rows with patients with no AIS codes

In [20]:
# find keys patient with no AIS codes
missing_pt = np.setdiff1d(test_demo.inc_key.unique(), mapped.inc_key.unique())

In [21]:
# loop through missing patients
for pt in missing_pt:
    
    # create datframe for patient
    pt_df = pd.DataFrame({'inc_key':[pt], 'aiscode':[0],'ais_predot':[0],'severity':[0],'chapter':[0],'region':[0]})
    
    mapped = mapped.append(pt_df, ignore_index=True).sort_values('inc_key').reset_index(drop=True)

## Store results

In [22]:
mapped.to_csv(results_ais_file, index=False)